In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR
from Envs.env_stocktrading import StockTradingEnv


%matplotlib inline

In [2]:
INDICATORS.append('vix')

In [3]:
hourly = True
sentiment = True
smoothed = False
seed = 0

path = ''
model_path = ''
model_name = f"seed{seed}"
SENTIMENT = []

In [4]:
if_using_a2c = True
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

Load Datafile

In [5]:
def prep(train):
    # Sort the DataFrame by the 'date' column
    train = train.sort_values(by='date')

    # Create a mapping of unique dates to their corresponding index
    date_mapping = {date: idx for idx, date in enumerate(train['date'].unique())}

    # Map the 'date' column to the date_mapping to get the new index
    train['index'] = train['date'].map(date_mapping)

    # Set the new index for the DataFrame
    train.set_index('index', inplace=True, drop=True)
    train = train.fillna(0)
    return train

In [6]:
if hourly:
    if smoothed:
        path = 'data/hourly/smoothed_testing_data_complete.csv'
        if sentiment:
            model_path =  'model/hourly/geglättetSentiment'
            SENTIMENT = ['stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 'random']
        else:
            model_path = 'model/hourly/geglättetNoSentiment'
            SENTIMENT = ['random']
    else:
        path = 'data/hourly/testing_data_complete.csv'
        if sentiment:
            model_path = 'model/hourly/withSentiment'
            SENTIMENT = ['stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 'random']
        else:
            model_path = 'model/hourly/noSentiment'
            SENTIMENT = ['random']
else:
    if smoothed:
        path = ''
        if sentiment:
            model_path = ''
            SENTIMENT = ['stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 'random']
        else:
            model_path = ''
            SENTIMENT = ['random']
    else:
        path = ''
        if sentiment:
            model_path = ''
            SENTIMENT = ['stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 'random']
        else:
            model_path = ''
            SENTIMENT = ['random']


In [7]:
#trade = pd.read_csv(path)
trade = pd.read_csv('training_data_complete.csv')
trade = prep(trade)
print(SENTIMENT)

['stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 'random']


Load the agents

In [8]:
trained_a2c = A2C.load(model_path + f"/a2c/{model_name}") if if_using_a2c else None
trained_ddpg = DDPG.load(model_path + f"/ddpg/{model_name}") if if_using_ddpg else None
trained_ppo = PPO.load(model_path + f"/ppo/{model_name}") if if_using_ppo else None
trained_td3 = TD3.load(model_path + f"/td3/{model_name}") if if_using_td3 else None
trained_sac = SAC.load(model_path + f"/sac/{model_name}") if if_using_sac else None

In [9]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 4 * stock_dimension + len(INDICATORS) * stock_dimension + len(SENTIMENT) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 541


In [10]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "sentiment_list": SENTIMENT,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "seed": seed,
}
e_trade_gym = StockTradingEnv(df=trade, risk_indicator_col='vix', **env_kwargs)

In [11]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym) if if_using_a2c else (None, None)

ValueError: Error: Unexpected observation shape (1, 541) for Box environment, please use (391,) or (n_env, 391) for the observation shape.

In [ ]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg,
    environment = e_trade_gym) if if_using_ddpg else (None, None)

In [ ]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo,
    environment = e_trade_gym) if if_using_ppo else (None, None)

In [ ]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3,
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [ ]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac,
    environment = e_trade_gym) if if_using_sac else (None, None)

In [ ]:
df_result_a2c = (
    df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
    if if_using_a2c
    else None
)
df_result_ddpg = (
    df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
    if if_using_ddpg
    else None
)
df_result_ppo = (
    df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
    if if_using_ppo
    else None
)
df_result_td3 = (
    df_account_value_td3.set_index(df_account_value_td3.columns[0])
    if if_using_td3
    else None
)
df_result_sac = (
    df_account_value_sac.set_index(df_account_value_sac.columns[0])
    if if_using_sac
    else None
)

result = pd.DataFrame(
    {
        "a2c": df_result_a2c["account_value"] if if_using_a2c else None,
        "ddpg": df_result_ddpg["account_value"] if if_using_ddpg else None,
        "ppo": df_result_ppo["account_value"] if if_using_ppo else None,
        "td3": df_result_td3["account_value"] if if_using_td3 else None,
        "sac": df_result_sac["account_value"] if if_using_sac else None,
    }
)

In [ ]:
result

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()